In [47]:
import neo4j
import pyvis
import numpy as np

In [48]:
uri = 'bolt://localhost:7687'
auth = ('neo4j', 'password')
db = neo4j.GraphDatabase.driver(uri, auth=auth)

In [49]:
db.execute_query(
    '''
    WITH "https://raw.githubusercontent.com/neo4j-graph-analytics/book/master/data/" AS base
    WITH base + "social-nodes.csv" AS uri
    LOAD CSV WITH HEADERS FROM uri AS row
    MERGE (:User {id: row.id})
    '''
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7f8bbfb03290>, keys=[])

In [50]:
db.execute_query(
    '''
    WITH "https://raw.githubusercontent.com/neo4j-graph-analytics/book/master/data/" AS base
    WITH base + "social-relationships.csv" AS uri
    LOAD CSV WITH HEADERS FROM uri AS row
    MATCH (source:User {id: row.src})
    MATCH (destination:User {id: row.dst})
    MERGE (source)-[:FOLLOWS]->(destination)
    '''
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7f8bbfb00a10>, keys=[])

In [51]:
users_df = db.execute_query(
    '''
    MATCH (s:User)-[f]->(d:User)
    RETURN s.id as s_id, d.id as d_id, TYPE(f) as rel_type
    ''',
    result_transformer_=neo4j.Result.to_df)

display(users_df)

,s_id,d_id,rel_type
0,Alice,Doug,FOLLOWS
1,Alice,Michael,FOLLOWS
2,Alice,Charles,FOLLOWS
3,Alice,Bridget,FOLLOWS
4,Bridget,Doug,FOLLOWS
5,Bridget,Alice,FOLLOWS
6,Bridget,Michael,FOLLOWS
7,Charles,Doug,FOLLOWS
8,Doug,Mark,FOLLOWS
9,Mark,Alice,FOLLOWS


In [52]:
user_graph_exists_result = db.execute_query(
    '''
    CALL gds.graph.exists('users')
    YIELD exists
    RETURN exists
    ''')

user_graph_exists = user_graph_exists_result.records[0].get('exists')

if user_graph_exists:
    db.execute_query(
        '''
        CALL gds.graph.drop('users')
        '''
    )

db.execute_query(
    '''
    MATCH (s:User)-[r:FOLLOWS]->(t:User)
    WITH gds.graph.project('users', s, t) AS g
    RETURN g
    ''')


EagerResult(records=[<Record g={'relationshipCount': 16, 'graphName': 'users', 'query': "\n    MATCH (s:User)-[r:FOLLOWS]->(t:User)\n    WITH gds.graph.project('users', s, t) AS g\n    RETURN g\n    ", 'projectMillis': 2369, 'configuration': {'readConcurrency': 4, 'undirectedRelationshipTypes': [], 'jobId': '406daa32-1b06-4030-b8a3-c10172353279', 'logProgress': True, 'query': "\n    MATCH (s:User)-[r:FOLLOWS]->(t:User)\n    WITH gds.graph.project('users', s, t) AS g\n    RETURN g\n    ", 'inverseIndexedRelationshipTypes': [], 'creationTime': neo4j.time.DateTime(2024, 2, 2, 12, 1, 40, 112162674, tzinfo=<UTC>)}, 'nodeCount': 9}>], summary=<neo4j._work.summary.ResultSummary object at 0x7f8bbfaf5890>, keys=['g'])

# Calculate Degree Centrality

In [53]:
# From the docs, the use of the 'UNDIRECTED' orientation uses both the in-degree and the outdegree to calculate the cenrtality degree.

degree_centrality_df = db.execute_query(
    '''
    CALL gds.degree.stream('users', { orientation: 'UNDIRECTED' })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).id as name, score as followers
    ORDER BY followers DESC, name DESC
    ''',
    result_transformer_=neo4j.Result.to_df)

display(degree_centrality_df)

,name,followers
0,Alice,7.0
1,Doug,6.0
2,Michael,5.0
3,Bridget,5.0
4,Mark,3.0
5,David,2.0
6,Charles,2.0
7,James,1.0
8,Amy,1.0


# Calculate Closeness Centrality

In [54]:
closeness_centrality_df = db.execute_query(
    '''
    CALL gds.closeness.stream('users', {useWassermanFaust: true})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).id as name, score as centrallity
    ORDER BY centrallity DESC, name DESC
    ''',
    result_transformer_=neo4j.Result.to_df)

display(closeness_centrality_df)

,name,centrallity
0,Doug,0.625000
1,Alice,0.390625
2,Mark,0.347222
3,Charles,0.312500
4,Michael,0.284091
5,Bridget,0.284091
6,Amy,0.166667
7,David,0.125000
8,James,0.000000


# Calculate Harmonic Centrality

In [55]:
closeness_centrality_df = db.execute_query(
    '''
    CALL gds.closeness.harmonic.stream('users')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).id as name, score as centrallity
    ORDER BY centrallity DESC, name DESC
    ''',
    result_transformer_=neo4j.Result.to_df)

display(closeness_centrality_df)

,name,centrallity
0,Doug,0.625000
1,Alice,0.479167
2,Michael,0.385417
3,Bridget,0.385417
4,Mark,0.375000
5,Charles,0.354167
6,Amy,0.187500
7,David,0.125000
8,James,0.000000


# Calculate Betweeness Centrality

In [56]:
betweeness_centrality_df = db.execute_query(
    '''
    CALL gds.betweenness.stream('users')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).id as name, score as centrallity
    ORDER BY centrallity DESC, name DESC
    ''',
    result_transformer_=neo4j.Result.to_df)

display(betweeness_centrality_df)

,name,centrallity
0,Alice,10.0
1,Mark,7.0
2,Doug,7.0
3,David,1.0
4,Michael,0.0
5,James,0.0
6,Charles,0.0
7,Bridget,0.0
8,Amy,0.0
